In [3]:
import tensorflow as tf

In [4]:
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding


In [5]:
class language_model():
    def __init__(self,data,params):
        self.data = data
        self.params = params
    
    def model(self):
        model = Sequential()
        model.add(Embedding(vocab_size, 10, input_length=max_length-1))
        model.add(LSTM(50))
        model.add(Dense(vocab_size, activation='softmax'))
        return model
    # generate a sequence from a language model
    def generate_seq(model, tokenizer, max_length, seed_text, n_words):
        in_text = seed_text
        # generate a fiselxed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([in_text])[0]
            # pre-pad sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
            # predict probabilities for each word
            yhat = model.predict_classes(encoded, verbose=0)
            # map predicted word index to word
            out_word = ''
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            # append to input
            in_text += ' ' + out_word
        return in_text

# source text
data = """ Jack and Jill went up the hill\n
        To fetch a pail of water\n
        Jack fell down and broke his crown\n
        And Jill came tumbling after\n """

params = {"activation":"softmax"}


lang_model = langauage_model(data,params)
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
# define model

print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 5))
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 3))
print(generate_seq(model, tokenizer, max_length-1, 'fell down', 5))
print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))

Vocabulary Size: 22
Total Sequences: 23
Max Sequence Length: 3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 10)             220       
_________________________________________________________________
lstm (LSTM)                  (None, 50)                12200     
_________________________________________________________________
dense (Dense)                (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
 - 1s - loss: 3.0910 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.0903 - acc: 0.0435
Epoch 3/500
 - 0s - loss: 3.0895 - acc: 0.0435
Epoch 4/500
 - 0s - loss: 3.0887 - acc: 0.1304
Epoch 5/500
 - 0s - loss: 3.0878 - acc: 0.1304
Epoch 6/500
 - 0s - loss: 3.0869 - acc: 0.1304
Epoch 7/500
 - 0s - loss: 3.0861 - acc: 0.1304
Epoch 8/500
 - 0s - loss: 3.0852 - acc: 0.1304
Epoch 9/500
 - 0s - loss: 3.0842 - acc: 0.1304
Epoch 10/500
 - 0s - loss: 3.0833 - acc: 0.1304
Epoch 11/500
 - 0s - loss: 3.0824 - acc: 0.1304
Epoch 12/500
 - 0s - loss: 3.0814 - acc: 0.1304
Epoch 13/500
 - 0s - loss: 3.0804 - acc: 0.1304
Epoch 14/500
 - 0s - loss: 3.0793 - acc: 0.1304
Epoch 15/500
 - 0s - loss: 3.0783 - acc: 0.1304
Epoch 16/500
 - 0s - loss: 3.0772 - acc: 0.1304
Epoch 17/500
 - 0s - loss: 3.0761 - acc: 0.1304
Epoch 18/500
 - 0s - loss: 3.0749 - acc: 0.1304
Epoch 19/500
 - 0s - loss: 3.0737 - acc: 0.1304
Epoch 20/500
 - 0s - loss: 3.0725 - acc: 0.1304
Epoch 21/500
 - 0s - loss: 3.0712 - acc: 0.13

Epoch 171/500
 - 0s - loss: 1.1620 - acc: 0.8261
Epoch 172/500
 - 0s - loss: 1.1441 - acc: 0.8261
Epoch 173/500
 - 0s - loss: 1.1262 - acc: 0.8261
Epoch 174/500
 - 0s - loss: 1.1084 - acc: 0.8261
Epoch 175/500
 - 0s - loss: 1.0906 - acc: 0.8696
Epoch 176/500
 - 0s - loss: 1.0728 - acc: 0.8696
Epoch 177/500
 - 0s - loss: 1.0551 - acc: 0.8696
Epoch 178/500
 - 0s - loss: 1.0375 - acc: 0.8696
Epoch 179/500
 - 0s - loss: 1.0198 - acc: 0.8696
Epoch 180/500
 - 0s - loss: 1.0023 - acc: 0.8696
Epoch 181/500
 - 0s - loss: 0.9848 - acc: 0.8696
Epoch 182/500
 - 0s - loss: 0.9674 - acc: 0.9130
Epoch 183/500
 - 0s - loss: 0.9500 - acc: 0.9130
Epoch 184/500
 - 0s - loss: 0.9327 - acc: 0.9130
Epoch 185/500
 - 0s - loss: 0.9155 - acc: 0.9130
Epoch 186/500
 - 0s - loss: 0.8985 - acc: 0.9130
Epoch 187/500
 - 0s - loss: 0.8814 - acc: 0.9130
Epoch 188/500
 - 0s - loss: 0.8646 - acc: 0.9130
Epoch 189/500
 - 0s - loss: 0.8479 - acc: 0.9130
Epoch 190/500
 - 0s - loss: 0.8313 - acc: 0.9130
Epoch 191/500
 - 0s 

Epoch 339/500
 - 0s - loss: 0.1020 - acc: 0.9565
Epoch 340/500
 - 0s - loss: 0.1016 - acc: 0.9565
Epoch 341/500
 - 0s - loss: 0.1011 - acc: 0.9565
Epoch 342/500
 - 0s - loss: 0.1007 - acc: 0.9565
Epoch 343/500
 - 0s - loss: 0.1003 - acc: 0.9565
Epoch 344/500
 - 0s - loss: 0.0999 - acc: 0.9565
Epoch 345/500
 - 0s - loss: 0.0995 - acc: 0.9565
Epoch 346/500
 - 0s - loss: 0.0990 - acc: 0.9565
Epoch 347/500
 - 0s - loss: 0.0986 - acc: 0.9565
Epoch 348/500
 - 0s - loss: 0.0983 - acc: 0.9565
Epoch 349/500
 - 0s - loss: 0.0979 - acc: 0.9565
Epoch 350/500
 - 0s - loss: 0.0975 - acc: 0.9565
Epoch 351/500
 - 0s - loss: 0.0971 - acc: 0.9565
Epoch 352/500
 - 0s - loss: 0.0967 - acc: 0.9565
Epoch 353/500
 - 0s - loss: 0.0964 - acc: 0.9565
Epoch 354/500
 - 0s - loss: 0.0960 - acc: 0.9565
Epoch 355/500
 - 0s - loss: 0.0957 - acc: 0.9565
Epoch 356/500
 - 0s - loss: 0.0953 - acc: 0.9565
Epoch 357/500
 - 0s - loss: 0.0950 - acc: 0.9565
Epoch 358/500
 - 0s - loss: 0.0947 - acc: 0.9565
Epoch 359/500
 - 0s 